**INITIALIZATION:**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.

In [1]:
#@ INITIALIZATION: 
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**LIBRARIES AND DEPENDENCIES:**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [13]:
#@ INSTALLING DEPENDENCIES: UNCOMMENT BELOW: 
# !pip install datasets transformers[sentencepiece]

In [14]:
#@ DOWNLOADING LIBRARIES AND DEPENDENCIES:
import torch
import transformers
import datasets
from datasets import load_dataset
from transformers import AdamW
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

#@ IGNORING WARNINGS: 
import warnings
warnings.filterwarnings("ignore")

**PROCESSING THE DATA:**

In [8]:
#@ PROCESSING THE DATA:
checkpoint = "bert-base-uncased"                                        # Initialization. 
tokenizer = AutoTokenizer.from_pretrained(checkpoint)                   # Initializing Tokenizer. 
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)  # Initializing Sequence Model. 
sequences = [
        "I've been waiting for a HuggingFace course my whole life",
        "This course is amazing!"
]                                                                       # Text Sequences. 
batch = tokenizer(sequences, padding=True, truncation=True, 
                  return_tensors="pt")                                  # Getting Batch of Tensors. 
batch["labels"] = torch.tensor([1, 1])                                  # Initializing Labels. 

#@ INITIALIZING MODEL TRAINING:
optimizer = AdamW(model.parameters())                                   # Initializing Optimizer. 
loss = model(**batch).loss                                              # Initializing Loss. 
loss.backward()                                                         # Initializing Back Propagation. 
optimizer.step()                                                        # Updating Parameters. 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

**GETTING THE DATASET:**
- In this notebook, we will use MRPC (Microsoft Research Paraphrase Corpus) dataset introduced by William B. Dolan and Chris Brockett. The dataset consist of 5801 pairs of sentences, with a label indicating if they are paraphrases or not. It is one of the 10 datasets composing the GLUE benchmark, which is an academic benchmark that is used to measure the performance of ML models across 10 different text classification tasks. 

In [17]:
#@ GETTING THE DATASET:
raw_datasets = load_dataset("glue", "mrpc")             # Getting MRPC Dataset. 
raw_datasets                                            # Inspecting Dataset. 

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [20]:
#@ INSPECTING TRAINING DATASET: 
raw_train_dataset = raw_datasets["train"]               # Training Dataset. 
raw_train_dataset[15]                                    # Inspection. 

{'idx': 16,
 'label': 0,
 'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
 'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .'}

In [19]:
#@ INSPECTING TYPE OF COLUMNS:
raw_train_dataset.features

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

**PREPROCESSING THE DATASET:**
- To preprocess the dataset, we will convert the text to numbers the model can make sense of, with the help of tokenizer. 